# Neural Style Transfer in Keras

## Import required libraries

In [1]:
import keras.backend as K
from keras.applications.vgg16 import preprocess_input, VGG16
from keras.preprocessing.image import load_img, img_to_array
from scipy.optimize import fmin_l_bfgs_b
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


## Set up files in the server

In [2]:
# !wget https://github.com/AparaV/artistic-style/raw/master/images/originals/river.jpg
# !wget https://github.com/AparaV/artistic-style/raw/master/images/originals/starry_night.jpg

In [3]:
# !mkdir img
# !mkdir output
# !mv *.jpg img/

In [4]:
!ls img

river.jpg  starry_night.jpg


In [5]:
cnt_img_path = 'img/river.jpg'
style_img_path = 'img/starry_night.jpg'
output_path = 'output/'

## Utilities

In [6]:
# def imsave(img, path):
#     img = post_process_image(img)
#     img = Image.fromarray(img)
#     img.save(path)

def imsave(img, path, target_size=(512, 512), postprocess=True):
    if postprocess:
        img = postprocess_array(img)
    img = Image.fromarray(img)
    img = img.resize(target_size)
    img.save(path)
    return img    

# def imread(path):
#     img = Image.open(path)
#     return np.asarray(img)
def imread(path, target_size=(512,512)):
    img = load_img(path=path, target_size=target_size)
    img = img_to_array(img)
    img = preprocess_input(np.expand_dims(img, axis=0))
    return img

def imread_tensor(path, target_size=(512,512)):
  # reads an image and returns a preprocessed tensor
    img = load_img(path=path, target_size=target_size)
    img = img_to_array(img)
    img = K.variable(preprocess_input(np.expand_dims(img, axis=0)), dtype='float32')
    return img

def implot(img):
    if type(img) == str: # read from path
        img = imread(img)
    plt.figure(figsize=(7,7))
    plt.axis('off')
    plt.imshow(img)

# def post_process_image(image):
#     image[:, :, 0] += 103.939
#     image[:, :, 1] += 116.779
#     image[:, :, 2] += 123.68
#     return np.clip(image[:, :, ::-1], 0, 255).astype('uint8')

def postprocess_array(x, target_size=(512, 512, 3)):
    if x.shape != target_size:
        x = x.reshape(target_size)

#     x[..., 0] = np.add(x[..., 0], 103.939, casting='unsafe')
#     x[..., 1] = np.add(x[..., 1], 116.779, casting='unsafe')
#     x[..., 2] = np.add(x[..., 2], 123.68, casting='unsafe')
    x[..., 0] += 103.939
    x[..., 1] += 116.779
    x[..., 2] += 123.68
    # 'BGR'->'RGB'
    x = x[..., ::-1]
    x = np.clip(x, 0, 255)
    x = x.astype('uint8')
    return x

## Implement Keras model

### Define important operations

In [7]:
def generate_canvas(mode='random', ref_image=None):
    ''' Generate a canvas and return a placeholder
    modes: random, from_ref
    ref_image: pass an img array or path if mode is 'from_ref'
    '''
    size = (512, 512, 3)

    if mode == 'random':
        img = np.random.randint(256, size=size)
    elif mode == 'from_ref':
        if type(ref_image) == str:
            img = load_img(path=ref_image, target_size=size)
            img = img_to_array(img)
        else:
            img = ref_image.copy()

    img = preprocess_input(np.expand_dims(img, axis=0))

    return img

In [8]:
def get_feature_maps(model, layers):
    '''get feature maps for given layers in the required format
    '''
    features = []

    for layer in layers:
        feat = model.get_layer(layer).output
        shape = K.shape(feat).eval(session=tf_session)
        M = shape[1] * shape[2]
        N = shape[-1]
        feat = K.transpose(K.reshape(feat, (M, N)))
        features.append(feat)

    return features   

![content loss](https://)

In [9]:
def content_loss(F, P):
    assert F.shape == P.shape
    loss = 0.5 * K.sum(K.square(F - P))
    return loss

In [10]:
def gram_matrix(matrix):
    return K.dot(matrix, K.transpose(matrix))

In [11]:
def style_loss(G, A):
    ''' Contribution of each layer to the total style loss
    '''
    assert G.shape == A.shape

#     N, M = K.shape(G).eval(session=tf_session)
    M, N = K.int_shape(G)[1], K.int_shape(G)[0]
    G, A = gram_matrix(G), gram_matrix(A)
    loss = 0.25 * K.sum(K.square(G - A)) / ((N ** 2) * (M ** 2))
    return loss

def total_style_loss(weights, Gs, As):
    ''' Get weighted total style loss
    '''
    loss = K.variable(0)

    for w, G, A in zip(weights, Gs, As):
        loss = loss + w * style_loss(G, A)

    return loss    

In [12]:
def total_loss(P, As, canvas_model, clayers, slayers, style_weights, alpha=1.0, beta=10000.0):
    ''' Get total loss
    params:
    x: generated image
    p: content image features
    a: style image features
    '''
    F = get_feature_maps(canvas_model, clayers)[0]
    Gs = get_feature_maps(canvas_model, slayers)

    closs = content_loss(F, P)
    sloss = total_style_loss(style_weights, Gs, As)

    loss = alpha * closs + beta * sloss  
    return loss  

###  Read images and define model

load all the images

In [13]:
target_size = (512, 512, 3)

In [14]:
cnt_img = imread_tensor(cnt_img_path)
style_img = imread_tensor(style_img_path)

setup VGG model and required configuraton

In [15]:
canvas_placeholder = K.placeholder(shape=(1,)+target_size)

In [16]:
# conv_net = VGG16(include_top=False, weights='imagenet', input_tensor=cnt_img)

In [17]:
cnt_model = VGG16(include_top=False, weights='imagenet', input_tensor=cnt_img)
style_model = VGG16(include_top=False, weights='imagenet', input_tensor=style_img)
canvas_model = VGG16(include_top=False, weights='imagenet', input_tensor=canvas_placeholder)

In [18]:
tf_session = K.get_session()

In [19]:
cnt_layers = ['block4_conv2']
style_layers = [
                'block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
]
Ws = [1.0 / float(len(style_layers))] * len(style_layers) # weights for each style layer

In [20]:
P = get_feature_maps(cnt_model, cnt_layers)[0]
As = get_feature_maps(style_model, style_layers)

X = generate_canvas('from_ref', cnt_img_path).flatten()

## Optimize the distance between content and style image

Define function to use with scipy optimizers. The total loss function is optimized using the **limited-memory BFGS** optimizer.

L-BFGS is a second order optimization method that works compared well to other popular methods when memory requirements can't be met.

In [21]:
epochs = 30
save_per_epoch = 10

In [22]:
# gimg - generated image in the canvas

step = 1

def calculate_loss(gimg):
    gimg = gimg.reshape((1,)+target_size)

    loss = total_loss(P, As, canvas_model, cnt_layers, style_layers, Ws)
    loss_func = K.function([canvas_model.input], [loss])
    return loss_func([gimg])[0].astype('float64')

def calculate_grad(gimg):
    gimg = gimg.reshape((1,)+target_size)

    loss = total_loss(P, As, canvas_model, cnt_layers, style_layers, Ws)
    gradients = K.gradients(loss, [canvas_model.input])
    grad_func = K.function([canvas_model.input], gradients)
    return grad_func([gimg])[0].flatten().astype('float64')

def callback(gimg):
    global step

    print(f'\rStep: {step}/{epochs}', end='')
    step += 1

    if (step % save_per_epoch) == 0 or (step == epochs):
        gimg = gimg.copy()
#         gimg = gimg.reshape(target_size)-+
        path = output_path + f'out_{step}.jpg'
        imsave(gimg, path)

In [23]:
# bounds = np.ndarray(shape=(X.shape[0], 2))
# bounds[:, 0] = -128.0
# bounds[:, 1] = 128.0

In [24]:
X_optim, _, info = fmin_l_bfgs_b(calculate_loss, X, fprime=calculate_grad, maxiter=epochs, callback=callback)

Step: 31/30

## View the generated image

In [ ]:
path = output_path + 'optimal.jpg'
imsave(XX_optim, path)

In [25]:
# img = X_optim.reshape(target_size)
# # img = postprocess_array(img)
# implot(img)

In [26]:
!ls output

out_10.jpg  out_20.jpg	out_30.jpg
